In [0]:
import numpy as np
from scipy.optimize import linprog

def nn2na(NN):
    # Get every location where exist an arc:
  arcs = np.argwhere(NN)
  NA = np.zeros([NN.shape[0], arcs.shape[0]]).astype(int)
  for i, arc in enumerate(arcs):
    NA[arc[0], i] = 1
    NA[arc[1], i] = -1

  arc_idx = [ (arc[0], arc[1]) for arc in arcs]
  return NA, arc_idx

# Shortest path Utils
def get_selected_arcs(arc_idxs, selected_arcs):
    arc = []
    for idx, i in enumerate(selected_arcs):
        if np.isclose(i, 1, rtol=1e-05, atol=1e-08, equal_nan=False): # Vecinity
            arc.append(arc_idxs[idx])
    return arc

def get_usage_string(arc_idxs, res_flow, capacity):
    return {arc: '%s/%s' % (flow, cap) for arc, flow, cap in zip(arc_idxs, res_flow, capacity)}

def get_min_cut(arc_idxs, np_res_flow, np_capacity):
    np_capacity = np.where(np_capacity == None, 999, np_capacity)

In [7]:
node_name = np.array((
    'P1a', 'P1b','P2a', 'P2b', 'P3a', 'P3b',
    'S1a', 'S1b','S2a', 'S2b',
    'V1a', 'V1b','V2a', 'V2b', 'V3a', 'V3b'))

# This matrix represent a Nodo-Nodo graph
matrix_node_node = np.array([
    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

NA, arc_idxs = nn2na(NN)

# 2 different Node-Arch matrices: constraits for plants and for sales points

# Plants: (rest of nodes equals 0)
Aub = NA.copy()
Aub[6:, :] = 0 
bub = np.array([20, 30, 10, 40, 30, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Sales point: (rest of nodes equals 0)
Aeq = NA.copy()
Aeq[:6, :] = 0
beq = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -30, -40, -10, -20, -20, -20])

cost_vector = np.array([100, 100, 200, 200, 150, 150, 150, 150, 200, 200, 100,
                           100, 100, 150, 200, 200, 150, 100, 100, 150, 200, 200, 150, 100])

bounds = tuple([(0, None) for arcs in range(0, NA.shape[1])])
    
# Optimize
res = linprog(cost_vector, A_eq=Aeq, b_eq=beq, A_ub=Aub, b_ub=bub, bounds=bounds, method="simplex")

print("Quantities: ")

for i in range(len(arc_idxs)):
    print(arc_idxs[i], ":", res.x[i].astype(int))


Quantities: 
(0, 6) : 20
(0, 8) : 0
(1, 7) : 30
(1, 9) : 0
(2, 6) : 10
(2, 8) : 0
(3, 7) : 40
(3, 9) : 0
(4, 6) : 30
(4, 8) : 0
(5, 7) : 10
(5, 9) : 0
(6, 10) : 30
(6, 12) : 10
(6, 14) : 20
(7, 11) : 40
(7, 13) : 20
(7, 15) : 20
(8, 10) : 0
(8, 12) : 0
(8, 14) : 0
(9, 11) : 0
(9, 13) : 0
(9, 15) : 0
